<a href="https://colab.research.google.com/github/TIN310/TTNT/blob/main/TTNT_Tuan3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Thuật giải Minimax **

khai báo các biến

In [52]:
import copy
import math
import random
import numpy
X = "X"
O = "O"
EMPTY = None
user = None
ai = None


khởi tạo 1 àn cờ 3x3 trống với mỗi EMPTY là 1 ô trống trên bàn cờ

In [53]:
def initial_state():
    return  [[EMPTY, EMPTY, EMPTY],
             [EMPTY, EMPTY, EMPTY],
             [EMPTY, EMPTY, EMPTY]]

Xác định lượt đi của người chơi (ai hoặc user )dựa trên trạng thái hiện tại của board
VD : nếu count = số chẳn sẽ trả về lượt đi của ai và lẻ sẽ trả về lượt đi của user

In [54]:
def player(board):
  count = 0
  for i in board:
    for j in i :
      if j :
        count += 1
  if count % 2 == 0 :
    return ai
  else :
    return user

xác định tất cả các bước đi hợp lệ (các ô trống ) , nếu ô đó = EMPTY (ô trống ) thì ô đó sẽ được thêm vào tập hợp res , cuối cùng hàm sẽ trả về tập hợp res qua return res nơi chứa tất cả các tọa độ  của các ô trống trên bàn cờ

In [55]:
def actions(board):
  res = set()
  board_len = len(board)
  for i in range(board_len):
    for j in range(board_len):
      if board[i][j] == EMPTY:
        res.add((i, j))
  return res

 Mô phỏng kết quả của việc thực hiện một hành động (nước đi) cụ thể trên một bàn cờ.
 Gọi hàm player(board) để xác định người chơi hiện tại đang có lượt đi. Đây có thể là ai hoặc user.
 result_board = copy.deepcopy(board): Tạo một bản sao sâu (deep copy) của bàn cờ hiện tại. Để đảm bảo bàn cờ gốc không bị thay đổi , điều này cho phép chúng ta thử ngiệm các nước đi mà không bị ảnh hưởng đến trạng thái ban đầu của trò chơi

In [56]:
def result (board , action ):
  curr_player = player(board)
  result_board = copy.deepcopy(board)
  (i,j) = action
  result_board[i][j] = curr_player
  return result_board

get_horizontal_winner(board) này được tạo ra để kiểm tra xem có bất kỳ hàng ngang nào trên bàn cờ có người thắng cuộc hay không.
1.   Khởi tạo một biến winner_val với giá trị None
dùng để lưu trữ ký hiệu của người thắng nếu thấy và vẫn giữ None nếu không có ai thắng
2.   winner_val = board[i][0] : hàm đang giả định rằng nếu có người thắng ở  hàng này thì sẽ là người đánh đầu tiên .



In [57]:
def get_horizontal_winner(board):
    board_len = len(board)
    for i in range(board_len):
       potential_winner = board[i][0]
       if potential_winner is None:
           continue

       is_winner_row = True
       for j in range(board_len):
          if board[i][j] != potential_winner:
            is_winner_row = False
            break

       if is_winner_row:
          return potential_winner

    return None

Hàm get_vertical_winner(board) này được tạo ra để kiểm tra xem có người thắng cuộc theo cột dọc trên bàn cờ hay không.


1.  winner_val = None: Biến này được khởi tạo để lưu trữ ký hiệu của người chơi thắng (X hoặc O) nếu tìm thấy một cột thắng.
2.   winner_val = board[0][i]: Khi bắt đầu kiểm tra một cột mới, hàm gán giá trị của ô đầu tiên trong cột đó (board[0][i]) cho winner_val. Đây là giả định rằng nếu có người thắng ở cột này, người đó phải là người đã điền vào ô đầu tiên.
3.   if board[j][i] != winner_val:: Trong vòng lặp này, giá trị của ô hiện tại (board[j][i]) được so sánh với winner_val (là giá trị của ô đầu tiên trong cột).
*Lưu ý :Nếu ô hiện tại khác với winner_val, điều đó có nghĩa là cột này không thể có người thắng với winner_val là biểu tượng của họ. Vì vậy, winner_val được đặt lại thành None.*




In [58]:
def get_vertical_winner(board):
# check vertically
  board_len = len(board)
  for i in range(board_len):
      potential_winner = board[0][i]
      if potential_winner is None:
          continue

      is_winner_col = True
      for j in range(board_len):
         if board[j][i] != potential_winner:
            is_winner_col = False
            break

      if is_winner_col:
         return potential_winner

  return None

get_diagonal_winner(board) này là để kiểm tra xem có người chơi nào đã thắng cuộc bằng cách tạo thành một đường thẳng trên một trong hai đường chéo của bàn cờ hay không.
1. winner_val = board[0][0]: Gán giá trị của ô đầu tiên trên đường chéo chính (ô (0,0)) cho winner_val. Điều này giả định rằng nếu có người thắng trên đường chéo này, thì người đó phải là người đã điền vào ô (0,0).
2. for i in range(board_len): Duyệt qua các ô trên đường chéo chính (các ô có tọa độ (i, i)).
3.   if board[i][i] != winner_val: winner_val = None: Nếu giá trị của ô hiện tại trên đường chéo không khớp với winner_val đã được thiết lập (từ ô (0,0)), thì đặt winner_val về None. Điều này có nghĩa là đường chéo này không có người thắng với ký hiệu ban đầu.





In [59]:
def get_diagonal_winner(board):
    winner_val = None
    board_len = len(board)
    potential_winner_main = None
    is_main_diagonal_winner = True
    for i in range(board_len):
        if potential_winner_main is None and board[i][i] != EMPTY:
            potential_winner_main = board[i][i]
        elif board[i][i] != potential_winner_main or board[i][i] == EMPTY:
            is_main_diagonal_winner = False
            break

    if is_main_diagonal_winner and potential_winner_main is not None:
        return potential_winner_main
    potential_winner_anti = None
    is_anti_diagonal_winner = True
    for i in range(board_len):
        j = board_len - 1 - i
        if potential_winner_anti is None and board[i][j] != EMPTY:
            potential_winner_anti = board[i][j]
        elif board[i][j] != potential_winner_anti or board[i][j] == EMPTY:
            is_anti_diagonal_winner = False
            break

    if is_anti_diagonal_winner and potential_winner_anti is not None:
        return potential_winner_anti

    return winner_val

Hàm winner(board) có ý nghĩa là kiểm tra toàn diện xem có người thắng cuộc trên bàn cờ hay chưa bằng cách kết hợp kết quả từ các hàm kiểm tra thắng theo hàng ngang, hàng dọc và đường chéo.

In [60]:
def winner(board):
  winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
  return winner_val

terminal(board): dùng để xác định xem ván cờ đã kết thúc hay chưa
1. if winner(board) != None:
Kiểm tra xem có người chơi nào đã thắng hay chưa.
2. return winner(board)
Nếu đã có người thắng thì sẽ trả về chính người thắng đó (ví dụ "X" hoặc "O").
3.  if j == EMPTY:
Kiểm tra xem ô hiện tại có phải là ô trống không.
*Nếu còn ít nhất một ô trống, nghĩa là trò chơi vẫn có thể tiếp tục.*




In [61]:
def terminal(board):
  if winner(board) != None:
    return winner(board)
  for i  in board:
     for j in i :
       if j == EMPTY :
         return False
  return True


utility(board):trả về giá trị đánh giá cuối cùng của ván cờ theo quy ước Minimax
1. winner_val = winner(board)
Gọi hàm winner(board) để kiểm tra xem có người chơi nào thắng ván cờ hay không.
2.  if winner_val == X:
Kiểm tra xem người thắng có phải là người chơi X hay không.
1.  return 1:
Nếu X thắng, hàm trả về
2.  elif winner_val == O:
Nếu X không thắng, kiểm tra tiếp xem người thắng có phải O hay không.





In [62]:
def utility (board):
  winner_val = winner(board)
  if winner_val == X:
    return 1
  elif winner_val == O :
    return -1
  return 0

maxValue(state):
Hàm này mô phỏng lượt đi của người chơi MAX (thường là X)
1.  if terminal(state):
Kiểm tra xem trạng thái hiện tại state có phải trạng thái kết thúc hay không.
2.  eturn utility(state)

Nếu trạng thái là kết thúc (terminal):

Gọi utility(state) để trả về giá trị đánh giá:
1 → X thắng

-1 → O thắng

0 → hòa

1. v = -math.inf
Khởi tạo v bằng âm vô cực.

2. for action in actions(state):
Duyệt qua tất cả các hành động hợp lệ mà người chơi hiện tại có thể thực hiện.



1.  v = max(v, minValue(result(state, action)))

Đây là dòng quan trọng nhất trong Minimax:

result(state, action) → tạo ra trạng thái mới sau khi MAX thực hiện nước đi.

minValue(...) → tính toán việc đối thủ (người chơi MIN) sẽ phản ứng tối ưu như thế nào.

max(v, ...) → chọn giá trị lớn nhất giữa:

Giá trị MAX hiện có v

Giá trị mà MIN có thể buộc MAX phải nhận sau nước đi đó

Ý nghĩa:Người chơi MAX luôn cố chọn nước đi sao cho:
 MIN phản ứng tối ưu theo hướng bất lợi, MAX vẫn nhận được giá trị tốt nhất có thể.





In [63]:
def maxValue(state):
 if terminal(state):
   return utility(state)
 v = -math.inf
 for action in actions(state):
     v = max(v, minValue(result(state, action)))
 return v

minimax(board):

Tìm nước đi tối ưu nhất cho người chơi hiện tại trên bàn cờ.
1. if terminal(state):

Kiểm tra xem trạng thái hiện tại state có phải trạng thái kết thúc hay không.

Nếu ván cờ đã kết thúc (có người thắng hoặc hòa) → không cần đi tiếp.

2. return utility(state):
Nếu trạng thái là kết thúc:

Gọi utility(state) để trả về giá trị đánh giá:

1 → X thắng

-1 → O thắng

0 → Hòa

3. v = math.inf:
Khởi tạo v bằng dương vô cực.

4. for action in actions(state):
Duyệt qua tất cả các hành động hợp lệ mà người chơi MIN có thể thực hiện.
actions(state) trả về danh sách các ô trống mà MIN có thể đặt quân vào.
Mỗi action là một nước đi khả thi.

5.  v = min(v, maxValue(result(state, action))):
result(state, action) sẽ tạo ra trạng thái mới sau khi MIN thực hiện nước đi.
Sau đó đến lượt MAX, nên gọi maxValue(...) để tính phản ứng tối ưu của MAX.
min(v, ...) → MIN chọn giá trị nhỏ nhất giữa:
Giá trị MIN hiện có (v)
Giá trị mà MAX có thể đạt được sau nước đi này
6. return v :
Sau khi xét hết tất cả hành động:
Trả về giá trị tối ưu mà MIN có thể đạt được từ trạng thái này.

In [71]:
def minimax(board):
    current_player = player(board)

    # X is MAX player
    if current_player == X:
        max_val = -math.inf
        best_move = None

        for action in actions(board):
            value = minValue(result(board, action))
            if value > max_val:
                max_val = value
                best_move = action

        return best_move

    # O is MIN player
    else:  # current_player == O
        min_val = math.inf
        best_move = None

        for action in actions(board):
            value = maxValue(result(board, action))
            if value < min_val:
                min_val = value
                best_move = action

        return best_move


Chương trình khởi tạo bàn cờ và hỏi người chơi muốn chơi X hay O.

AI sẽ chơi nước đi tối ưu theo Minimax.

Người chơi nhập vị trí ô muốn đánh, chương trình kiểm tra hợp lệ.

Vòng lặp tiếp tục cho đến khi game kết thúc (thắng , thua hoặc hòa).

Bàn cờ được hiển thị sau mỗi nước đi, thông báo kết quả cuối cùng khi game kết thúc.

In [73]:
if __name__ == "__main__":
    board = initial_state()
    ai_turn = False

    print("Choose a player (X or O):")
    user = input().strip().upper()

    if user == "X":
        ai = "O"
    else:
        ai = "X"

    while True:
        game_over = terminal(board)
        current_player = player(board)

        # Nếu game kết thúc
        if game_over:
            win = winner(board)   # không đặt tên biến là "winner" để tránh đè hàm winner()

            if win is None:
                print("Game Over: Tie.")
            else:
                print(f"Game Over: {win} wins.")
            break

        # ----------- Lượt AI -----------
        if current_player == ai:
            print("\nAI is thinking...")
            move = minimax(board)
            board = result(board, move)
            print("AI moved:")
            print(numpy.array(board))

        # ----------- Lượt người chơi -----------
        else:
            print("\nEnter the position to move (row, col):")
            while True:
                try:
                    i = int(input("Row (0, 1, or 2): "))
                    j = int(input("Col (0, 1, or 2): "))

                    # Validate input ranges
                    if 0 <= i < len(board) and 0 <= j < len(board):
                        if board[i][j] == EMPTY:
                            board = result(board, (i, j))
                            print(numpy.array(board))
                            break # Exit the loop if move is valid
                        else:
                            print("Invalid move! Cell is not empty. Try again.")
                    else:
                        print(f"Invalid input. Row and column must be between 0 and {len(board)-1}. Try again.")
                except ValueError:
                    print("Invalid input. Please enter a number. Try again.")


Choose a player (X or O):
O

AI is thinking...
AI moved:
[[None 'X' None]
 [None None None]
 [None None None]]

Enter the position to move (row, col):
Row (0, 1, or 2): 3
Col (0, 1, or 2): 3
Invalid input. Row and column must be between 0 and 2. Try again.


KeyboardInterrupt: Interrupted by user